# extracttion des données de logement de l'ademe via l'api

## I- extractions des données existants

In [2]:
import requests
import pandas as pd
import concurrent.futures
import os
import time


In [4]:
import requests
import pandas as pd
import concurrent.futures
import os
import time

# === CONFIG ===
BASE_URL = "https://data.ademe.fr/data-fair/api/v1/datasets/dpe03existant/lines"
CODES_POSTAUX_FILE = "adresses-69.csv"   # fichier contenant une colonne "code_postal"
OUTPUT_FILE = "data_existants_69.csv"

# Charger les codes postaux du département
code_postaux_df = pd.read_csv(CODES_POSTAUX_FILE, dtype=str, sep=';')
code_postals = code_postaux_df['code_postal'].unique().tolist()

# Colonnes à extraire
listeColumn = "configuration_installation_chauffage_n1,conso_chauffage_installation_chauffage_n1,type_generateur_n1_ecs_n1,numero_voie_ban,score_ban,surface_habitable_immeuble," \
"conso_auxiliaires_ep,deperditions_murs,cout_eclairage,conso_auxiliaires_ef,statut_geocodage,ventilation_posterieure_2012,cout_chauffage,conso_5_usages_par_m2_ep,date_etablissement_dpe," \
"conso_ecs_ef_energie_n1,conso_ecs_ef_energie_n2,emission_ges_chauffage,description_installation_chauffage_n1,conso_5_usages_par_m2_ef,cout_ecs_energie_n2,conso_chauffage_ef_energie_n1," \
"conso_chauffage_ef_energie_n2,qualite_isolation_menuiseries,cout_total_5_usages_energie_n2,date_reception_dpe,cout_total_5_usages_energie_n1,cout_ecs_energie_n1,qualite_isolation_plancher_bas," \
"modele_dpe,qualite_isolation_enveloppe,conso_chauffage_generateur_n1_installation_n1,type_energie_n1,emission_ges_eclairage,type_energie_n2,code_postal_ban,emission_ges_ecs,conso_5_usages_ef_energie_n2," \
"conso_5_usages_ef,conso_5_usages_ef_energie_n1,code_insee_ban,deperditions_planchers_bas,conso_5_usages_ep,date_fin_validite_dpe,deperditions_enveloppe,code_region_ban,volume_stockage_generateur_n1_ecs_n1," \
"surface_chauffee_installation_chauffage_n1,version_dpe,besoin_ecs,coordonnee_cartographique_x_ban,type_generateur_chauffage_principal,type_energie_principale_ecs,apport_solaire_saison_chauffe,adresse_ban," \
"nombre_appartement,deperditions_renouvellement_air,_rand,surface_habitable_desservie_par_installation_ecs_n1,production_electricite_pv_kwhep_par_an,type_installation_chauffage,nombre_niveau_logement," \
"surface_habitable_logement,cout_ecs,type_installation_ecs_n1,emission_ges_5_usages_energie_n1,emission_ges_5_usages_energie_n2,apport_interne_saison_froide,emission_ges_5_usages_par_m2," \
"description_generateur_chauffage_n1_installation_n1,qualite_isolation_plancher_haut_comble_perdu,apport_interne_saison_chauffe,apport_solaire_saison_froide,type_generateur_n1_installation_n1," \
"nombre_logements_desservis_par_installation_ecs_n1,complement_adresse_logement,cout_auxiliaires,type_emetteur_installation_chauffage_n1,besoin_chauffage,configuration_installation_ecs_n1,description_installation_ecs_n1," \
"classe_inertie_batiment,deperditions_ponts_thermiques,type_generateur_chauffage_principal_ecs,emission_ges_refroidissement,hauteur_sous_plafond,conso_chauffage_ef,nom_commune_ban,annee_construction,_geopoint,date_visite_diagnostiqueur," \
"type_batiment,periode_construction,type_installation_ecs,conso_ecs_ep,conso_ecs_ef,emission_ges_5_usages,date_derniere_modification_dpe,etiquette_ges,identifiant_ban,deperditions_baies_vitrees,type_energie_generateur_n1_ecs_n1,ubat_w_par_m2_k," \
"numero_etage_appartement,nom_commune_brut,conso_ef_installation_ecs_n1,etiquette_dpe,description_generateur_n1_ecs_n1,code_departement_ban,type_installation_chauffage_n1,methode_application_dpe,adresse_brut,cout_total_5_usages,categorie_enr," \
"conso_refroidissement_ef,conso_eclairage_ef,deperditions_planchers_hauts,zone_climatique,conso_ef_generateur_n1_ecs_n1,emission_ges_ecs_energie_n1,emission_ges_ecs_energie_n2,cout_refroidissement,conso_chauffage_ep,conso_eclairage_ep,usage_generateur_n1_installation_n1," \
"nom_rue_ban,qualite_isolation_murs,type_installation_solaire_n1,classe_altitude,conso_refroidissement_ep,type_energie_principale_chauffage,numero_dpe,_i,besoin_refroidissement,emission_ges_chauffage_energie_n2,emission_ges_chauffage_energie_n1,cout_chauffage_energie_n2," \
"deperditions_portes,cout_chauffage_energie_n1,coordonnee_cartographique_y_ban,type_energie_generateur_n1_installation_n1,code_postal_brut,emission_ges_auxiliaires,usage_generateur_n1_ecs_n1"

# Supprimer ancien fichier s'il existe
if os.path.exists(OUTPUT_FILE):
    os.remove(OUTPUT_FILE)

all_results = []

def fetch_data_smart(code_postal, etiquette=None, start_date=None, end_date=None):
    results = []
    size = 1000
    page = 1

    # Construction du filtre q
    q_parts = [f"code_postal_ban:{code_postal}"]
    if etiquette:
        q_parts.append(f"etiquette_dpe:{etiquette}")
    if start_date and end_date:
        q_parts.append(f"date_reception_dpe:[{start_date} TO {end_date}]")
    q_filter = " AND ".join(q_parts)

    print(f"\n--- Début téléchargement : {q_filter} ---")

    # Première requête pour connaître le total
    params = {"page": 1, "size": size, "qs": q_filter, "select": listeColumn, "q_fields": "code_postal_ban,etiquette_dpe,date_reception_dpe"}
    response = requests.get(BASE_URL, params=params)
    if response.status_code != 200:
        print(f" Erreur {response.status_code} pour {q_filter}")
        return results

    data = response.json()
    total = data.get("total", 0)
    print(f"Total estimé pour {q_filter} : {total}")

    # Si total > 1000 et pas encore filtré par etiquette
    if total > 10000 and etiquette is None:
        print(f"Nombre trop élevé ({total}), découpage par étiquette...")
        etiquettes = ["A", "B", "C", "D", "E", "F", "G"]
        for etiq in etiquettes:
            results.extend(fetch_data_smart(code_postal, etiquette=etiq))
        return results

    # Si total > 1000 et déjà filtré par etiquette mais pas par date
    if total > 10000 and etiquette is not None and start_date is None:
        print(f"Nombre trop élevé ({total}) pour {code_postal} et étiquette {etiquette}, découpage par année...")
        for year in range(2021, 2025):  # exemple années 2021 à 2024
            year_start = f"{year}-01-01"
            year_end = f"{year}-12-31"
            results.extend(fetch_data_smart(code_postal, etiquette, year_start, year_end))
        return results

    # Sinon récupération normale par page
    while True:
        params["page"] = page
        response = requests.get(BASE_URL, params=params)
        if response.status_code != 200:
            print(f" Erreur {response.status_code} page {page} pour {q_filter}")
            break

        page_results = response.json().get("results", [])
        if not page_results:
            print(f" Aucun résultat à la page {page}, arrêt.")
            break

        results.extend(page_results)
        print(f"{q_filter} Page {page} : {len(page_results)} résultats — cumul : {len(results)}/{total}")

        if len(page_results) < size:
            print(f" Dernière page atteinte pour {q_filter}")
            break

        page += 1
        time.sleep(3)

    print(f"--- Fin téléchargement : {q_filter} — {len(results)} DPE récupérés ---\n")
    return results

all_results = []
for cp in code_postals:
    all_results.extend(fetch_data_smart(cp))

df = pd.DataFrame(all_results)
df.to_csv("data_existants_69.csv", index=False, encoding='utf-8')
print(f"Export terminé : data_existants_69.csv ({len(df)} lignes)")


#df = pd.DataFrame(all_results)
#df.to_csv("data_existants_69.csv", index=False, encoding='utf-8')
#print(f" Export terminé : data_existants_69.csv ({len(df)} lignes)")



--- Début téléchargement : code_postal_ban:69790 ---
Total estimé pour code_postal_ban:69790 : 200
code_postal_ban:69790 Page 1 : 200 résultats — cumul : 200/200
 Dernière page atteinte pour code_postal_ban:69790
--- Fin téléchargement : code_postal_ban:69790 — 200 DPE récupérés ---


--- Début téléchargement : code_postal_ban:69170 ---
Total estimé pour code_postal_ban:69170 : 2928
code_postal_ban:69170 Page 1 : 1000 résultats — cumul : 1000/2928
code_postal_ban:69170 Page 2 : 1000 résultats — cumul : 2000/2928
code_postal_ban:69170 Page 3 : 928 résultats — cumul : 2928/2928
 Dernière page atteinte pour code_postal_ban:69170
--- Fin téléchargement : code_postal_ban:69170 — 2928 DPE récupérés ---


--- Début téléchargement : code_postal_ban:69250 ---
Total estimé pour code_postal_ban:69250 : 3222
code_postal_ban:69250 Page 1 : 1000 résultats — cumul : 1000/3222
code_postal_ban:69250 Page 2 : 1000 résultats — cumul : 2000/3222
code_postal_ban:69250 Page 3 : 1000 résultats — cumul : 300

In [3]:
data_existants = pd.read_csv("data_existants_69.csv")
data_existants.shape

C:\Users\Surface\AppData\Local\Temp\ipykernel_4856\1118627378.py:1: DtypeWarning: Columns (38) have mixed types. Specify dtype option on import or set low_memory=False.
  data_existants = pd.read_csv("data_existants_69.csv")


(407945, 145)

In [23]:
data_existants['date_reception_dpe'].sort_values().unique()

array(['2021-07-01', '2021-07-02', '2021-07-03', ..., '2025-10-11',
       '2025-10-12', '2025-10-13'], shape=(1565,), dtype=object)

In [21]:
data_existants.head()

,configuration_installation_chauffage_n1,conso_chauffage_installation_chauffage_n1,type_generateur_n1_ecs_n1,score_ban,conso_auxiliaires_ep,deperditions_murs,cout_eclairage,conso_auxiliaires_ef,statut_geocodage,ventilation_posterieure_2012,...,emission_ges_auxiliaires,usage_generateur_n1_ecs_n1,_score,categorie_enr,numero_voie_ban,nom_rue_ban,type_installation_chauffage,type_installation_ecs,nombre_appartement,surface_habitable_immeuble
0,Installation de chauffage simple,23676.5,Chaudière fioul basse température 1991-2015,0.36,2136.4,80.0,66.4,928.9,adresse géocodée ban à l'adresse,0,...,59.4,chauffage + ecs,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Installation de chauffage simple,24049.0,Ballon électrique à accumulation vertical Autr...,0.25,882.8,135.0,33.9,383.8,adresse géocodée ban à l'adresse,0,...,24.6,ecs,NaN,chauffage au bois,NaN,NaN,NaN,NaN,NaN,NaN
2,Installation de chauffage avec en appoint un i...,1.0,Ballon électrique à accumulation vertical Caté...,0.27,1.0,148.6,15.3,0.0,adresse géocodée ban à l'adresse,0,...,0.0,chauffage,NaN,chauffage au bois,290,Chemin du Vernay,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,0.27,1734.7,50.7,63.1,754.2,adresse géocodée ban à l'adresse,0,...,48.3,NaN,NaN,pompe à chaleur,NaN,NaN,NaN,NaN,NaN,NaN
4,Installation de chauffage simple,1.0,Ballon électrique à accumulation vertical Autr...,0.41,866.1,169.3,48.8,376.6,adresse géocodée ban à l'adresse,0,...,24.1,chauffage,NaN,NaN,2101,Route du Saint-Rigaud,NaN,NaN,NaN,NaN


In [51]:
data_existants['annee_construction'].sort_values().unique()

array([1460., 1516., 1522., 1525., 1550., 1580., 1595., 1600., 1610.,
       1621., 1622., 1640., 1650., 1656., 1657., 1660., 1663., 1675.,
       1681., 1686., 1690., 1700., 1720., 1731., 1732., 1740., 1741.,
       1750., 1751., 1760., 1765., 1770., 1779., 1780., 1783., 1785.,
       1789., 1790., 1793., 1800., 1802., 1810., 1812., 1815., 1820.,
       1822., 1824., 1825., 1827., 1830., 1832., 1835., 1836., 1840.,
       1842., 1845., 1846., 1848., 1849., 1850., 1853., 1855., 1856.,
       1858., 1860., 1861., 1862., 1863., 1864., 1865., 1867., 1868.,
       1869., 1870., 1871., 1872., 1873., 1875., 1876., 1878., 1880.,
       1882., 1883., 1884., 1885., 1886., 1887., 1888., 1889., 1890.,
       1891., 1892., 1893., 1894., 1895., 1896., 1897., 1898., 1899.,
       1900., 1901., 1902., 1903., 1904., 1905., 1906., 1907., 1908.,
       1909., 1910., 1911., 1912., 1913., 1914., 1915., 1916., 1917.,
       1918., 1919., 1920., 1921., 1922., 1923., 1924., 1925., 1926.,
       1927., 1928.,

In [61]:
data_existants.loc[data_existants['annee_construction'] > 2015, 'annee_construction'] \
    .value_counts().sort_index()


annee_construction
2016.0    3007
2017.0    2253
2018.0    2557
2019.0    1894
2020.0    1450
2021.0    1801
2022.0     335
2023.0     341
2024.0      92
2025.0      92
Name: count, dtype: int64

In [11]:
data_existants['annee_construction'].sort_values().unique()

array([1460., 1516., 1522., 1525., 1550., 1580., 1595., 1600., 1610.,
       1621., 1622., 1640., 1650., 1656., 1657., 1660., 1663., 1675.,
       1681., 1686., 1690., 1700., 1720., 1731., 1732., 1740., 1741.,
       1750., 1751., 1760., 1765., 1770., 1779., 1780., 1783., 1785.,
       1789., 1790., 1793., 1800., 1802., 1810., 1812., 1815., 1820.,
       1822., 1824., 1825., 1827., 1830., 1832., 1835., 1836., 1840.,
       1842., 1845., 1846., 1848., 1849., 1850., 1853., 1855., 1856.,
       1858., 1860., 1861., 1862., 1863., 1864., 1865., 1867., 1868.,
       1869., 1870., 1871., 1872., 1873., 1875., 1876., 1878., 1880.,
       1882., 1883., 1884., 1885., 1886., 1887., 1888., 1889., 1890.,
       1891., 1892., 1893., 1894., 1895., 1896., 1897., 1898., 1899.,
       1900., 1901., 1902., 1903., 1904., 1905., 1906., 1907., 1908.,
       1909., 1910., 1911., 1912., 1913., 1914., 1915., 1916., 1917.,
       1918., 1919., 1920., 1921., 1922., 1923., 1924., 1925., 1926.,
       1927., 1928.,

## 2-  données des logements neufs

In [ ]:
# recuperation des noms de variables des logements neufs
import json
#data = json.loads('variables.json')
file = 'variables_logement_neufs.json'
with open(file) as train_file:
    data = json.load(train_file)
variables = pd.json_normalize(data['results'])
res = list(variables.columns.values)
colonnes = ",".join(res)
colonnes

'emission_ges_5_usages_energie_n1,appartement_non_visite,score_ban,emission_ges_5_usages_par_m2,surface_habitable_immeuble,conso_auxiliaires_ep,complement_adresse_logement,cout_eclairage,cout_auxiliaires,conso_auxiliaires_ef,statut_geocodage,ventilation_posterieure_2012,cout_chauffage,conso_5_usages_par_m2_ep,emission_ges_refroidissement,date_etablissement_dpe,conso_ecs_ef_energie_n1,hauteur_sous_plafond,conso_chauffage_ef,emission_ges_chauffage,nom_commune_ban,conso_5_usages_par_m2_ef,_geopoint,date_visite_diagnostiqueur,type_batiment,conso_chauffage_ef_energie_n1,qualite_isolation_menuiseries,conso_ecs_ep,date_reception_dpe,cout_total_5_usages_energie_n1,cout_ecs_energie_n1,qualite_isolation_plancher_bas,conso_ecs_ef,emission_ges_5_usages,date_derniere_modification_dpe,etiquette_ges,identifiant_ban,modele_dpe,qualite_isolation_enveloppe,ubat_w_par_m2_k,type_energie_n1,emission_ges_eclairage,nom_commune_brut,code_postal_ban,etiquette_dpe,emission_ges_ecs,conso_5_usages_ef,conso_5_usag

In [ ]:
import requests
import pandas as pd
import concurrent.futures
import os
import time

# === CONFIG ===
BASE_URL = "https://data.ademe.fr/data-fair/api/v1/datasets/dpe02neuf/lines"
CODES_POSTAUX_FILE = "adresses-69.csv"   # fichier contenant une colonne "code_postal"
OUTPUT_FILE = "data_neufs_69.csv"

# Charger les codes postaux du département
code_postaux_df = pd.read_csv(CODES_POSTAUX_FILE, dtype=str, sep=';')
code_postals = code_postaux_df['code_postal'].unique().tolist()

# Colonnes à extraire
listeColumn = "emission_ges_5_usages_energie_n1,appartement_non_visite,score_ban,emission_ges_5_usages_par_m2,surface_habitable_immeuble,conso_auxiliaires_ep,complement_adresse_logement,cout_eclairage," \
"cout_auxiliaires,conso_auxiliaires_ef,statut_geocodage,ventilation_posterieure_2012,cout_chauffage,conso_5_usages_par_m2_ep,emission_ges_refroidissement,date_etablissement_dpe,conso_ecs_ef_energie_n1," \
"hauteur_sous_plafond,conso_chauffage_ef,emission_ges_chauffage,nom_commune_ban,conso_5_usages_par_m2_ef,_geopoint,date_visite_diagnostiqueur,type_batiment,conso_chauffage_ef_energie_n1,qualite_isolation_menuiseries," \
"conso_ecs_ep,date_reception_dpe,cout_total_5_usages_energie_n1,cout_ecs_energie_n1,qualite_isolation_plancher_bas,conso_ecs_ef,emission_ges_5_usages,date_derniere_modification_dpe,etiquette_ges,identifiant_ban,modele_dpe," \
"qualite_isolation_enveloppe,ubat_w_par_m2_k,type_energie_n1,emission_ges_eclairage,nom_commune_brut,code_postal_ban,etiquette_dpe,emission_ges_ecs,conso_5_usages_ef,conso_5_usages_ef_energie_n1,code_departement_ban,code_insee_ban," \
"nombre_niveau_immeuble,conso_5_usages_ep,date_fin_validite_dpe,methode_application_dpe,adresse_brut,code_region_ban,cout_total_5_usages,categorie_enr,conso_refroidissement_ef,conso_eclairage_ef,emission_ges_ecs_energie_n1,cout_refroidissement," \
"conso_chauffage_ep,version_dpe,conso_eclairage_ep,nom_rue_ban,coordonnee_cartographique_x_ban,qualite_isolation_murs,conso_refroidissement_ep,type_energie_principale_chauffage,numero_dpe,_i,type_energie_principale_ecs,emission_ges_chauffage_energie_n1," \
"adresse_ban,nombre_appartement,cout_chauffage_energie_n1,_rand,coordonnee_cartographique_y_ban,code_postal_brut,production_electricite_pv_kwhep_par_an,emission_ges_auxiliaires,nombre_niveau_logement,surface_habitable_logement,cout_ecs"

# Supprimer ancien fichier s'il existe
if os.path.exists(OUTPUT_FILE):
    os.remove(OUTPUT_FILE)

all_results = []

def fetch_data_smart(code_postal, etiquette=None, start_date=None, end_date=None):
    results = []
    size = 1000
    page = 1

    # Construction du filtre q
    q_parts = [f"code_postal_ban:{code_postal}"]
    if etiquette:
        q_parts.append(f"etiquette_dpe:{etiquette}")
    if start_date and end_date:
        q_parts.append(f"date_reception_dpe:[{start_date} TO {end_date}]")
    q_filter = " AND ".join(q_parts)

    print(f"\n--- Début téléchargement : {q_filter} ---")

    # Première requête pour connaître le total
    params = {"page": 1, "size": size, "qs": q_filter, "select": listeColumn, "q_fields": "code_postal_ban,etiquette_dpe,date_reception_dpe"}
    response = requests.get(BASE_URL, params=params)
    if response.status_code != 200:
        print(f" Erreur {response.status_code} pour {q_filter}")
        return results

    data = response.json()
    total = data.get("total", 0)
    print(f"Total estimé pour {q_filter} : {total}")

    # Si total > 1000 et pas encore filtré par etiquette
    if total > 10000 and etiquette is None:
        print(f"Nombre trop élevé ({total}), découpage par étiquette...")
        etiquettes = ["A", "B", "C", "D", "E", "F", "G"]
        for etiq in etiquettes:
            results.extend(fetch_data_smart(code_postal, etiquette=etiq))
        return results

    # Si total > 1000 et déjà filtré par etiquette mais pas par date
    if total > 10000 and etiquette is not None and start_date is None:
        print(f"Nombre trop élevé ({total}) pour {code_postal} et étiquette {etiquette}, découpage par année...")
        for year in range(2021, 2025):  # exemple années 2021 à 2024
            year_start = f"{year}-01-01"
            year_end = f"{year}-12-31"
            results.extend(fetch_data_smart(code_postal, etiquette, year_start, year_end))
        return results

    # Sinon récupération normale par page
    while True:
        params["page"] = page
        response = requests.get(BASE_URL, params=params)
        if response.status_code != 200:
            print(f" Erreur {response.status_code} page {page} pour {q_filter}")
            break

        page_results = response.json().get("results", [])
        if not page_results:
            print(f" Aucun résultat à la page {page}, arrêt.")
            break

        results.extend(page_results)
        print(f"{q_filter} Page {page} : {len(page_results)} résultats — cumul : {len(results)}/{total}")

        if len(page_results) < size:
            print(f" Dernière page atteinte pour {q_filter}")
            break

        page += 1
        time.sleep(3)

    print(f"--- Fin téléchargement : {q_filter} — {len(results)} DPE récupérés ---\n")
    return results

all_results = []
for cp in code_postals:
    all_results.extend(fetch_data_smart(cp))

df = pd.DataFrame(all_results)
df.to_csv("data_neufs_69.csv", index=False, encoding='utf-8')
print(f"Export terminé : data_neufs_69.csv ({len(df)} lignes)")


--- Début téléchargement : code_postal_ban:69790 ---
Total estimé pour code_postal_ban:69790 : 1
code_postal_ban:69790 Page 1 : 1 résultats — cumul : 1/1
 Dernière page atteinte pour code_postal_ban:69790
--- Fin téléchargement : code_postal_ban:69790 — 1 DPE récupérés ---


--- Début téléchargement : code_postal_ban:69170 ---
Total estimé pour code_postal_ban:69170 : 233
code_postal_ban:69170 Page 1 : 233 résultats — cumul : 233/233
 Dernière page atteinte pour code_postal_ban:69170
--- Fin téléchargement : code_postal_ban:69170 — 233 DPE récupérés ---


--- Début téléchargement : code_postal_ban:69250 ---
Total estimé pour code_postal_ban:69250 : 482
code_postal_ban:69250 Page 1 : 482 résultats — cumul : 482/482
 Dernière page atteinte pour code_postal_ban:69250
--- Fin téléchargement : code_postal_ban:69250 — 482 DPE récupérés ---


--- Début téléchargement : code_postal_ban:69380 ---
Total estimé pour code_postal_ban:69380 : 560
code_postal_ban:69380 Page 1 : 560 résultats — cumul

In [5]:
data_neufs = pd.read_csv("data_neufs_69.csv")
data_neufs.head()

C:\Users\Surface\AppData\Local\Temp\ipykernel_4856\2118894399.py:1: DtypeWarning: Columns (49) have mixed types. Specify dtype option on import or set low_memory=False.
  data_neufs = pd.read_csv("data_neufs_69.csv")


,emission_ges_5_usages_energie_n1,appartement_non_visite,score_ban,emission_ges_5_usages_par_m2,surface_habitable_immeuble,conso_auxiliaires_ep,complement_adresse_logement,cout_eclairage,cout_auxiliaires,conso_auxiliaires_ef,...,cout_chauffage_energie_n1,_rand,coordonnee_cartographique_y_ban,code_postal_brut,production_electricite_pv_kwhep_par_an,emission_ges_auxiliaires,nombre_niveau_logement,surface_habitable_logement,cout_ecs,_score
0,27.5,0.0,0.22,1.7,116.6,163.3,1-Mr et Mme SIMON Florian et Pauline-1-LogZone,45.0,14.0,71.0,...,350.0,794968,6569539.78,78490,0.0,4.5,1.0,116.6,95.0,NaN
1,1251.7,NaN,0.96,18.9,NaN,20.9,NaN,132.9,3.3,9.1,...,374.4,124952,6533498.54,69170,0.0,0.6,1.0,67.8,415.7,NaN
2,1181.5,NaN,0.96,20.4,NaN,20.9,NaN,115.4,3.3,9.1,...,371.6,242969,6533498.54,69170,0.0,0.6,1.0,59.0,392.8,NaN
3,29.6,0.0,0.62,2.2,144.1,121.0,1-Batiment n 1-1-LogZone,54.0,9.0,52.6,...,562.0,255810,6542759.44,69170,0.0,3.4,1.0,144.1,133.0,NaN
4,1122.7,NaN,0.96,22.1,NaN,20.9,NaN,101.2,3.3,9.1,...,328.8,165190,6533498.54,69170,0.0,0.6,1.0,51.7,403.1,NaN


In [6]:
data_neufs['date_reception_dpe'].sort_values().unique()

array(['2021-07-01', '2021-07-05', '2021-07-09', ..., '2025-10-16',
       '2025-10-17', '2025-10-20'], shape=(1137,), dtype=object)

## III- Manipulation des données de l'ademe

In [39]:
# variables des logements anciens
for col in data_existants.columns:
    print(col)

configuration_installation_chauffage_n1
conso_chauffage_installation_chauffage_n1
type_generateur_n1_ecs_n1
score_ban
conso_auxiliaires_ep
deperditions_murs
cout_eclairage
conso_auxiliaires_ef
statut_geocodage
ventilation_posterieure_2012
cout_chauffage
conso_5_usages_par_m2_ep
date_etablissement_dpe
conso_ecs_ef_energie_n1
conso_ecs_ef_energie_n2
emission_ges_chauffage
description_installation_chauffage_n1
conso_5_usages_par_m2_ef
cout_ecs_energie_n2
conso_chauffage_ef_energie_n1
conso_chauffage_ef_energie_n2
qualite_isolation_menuiseries
cout_total_5_usages_energie_n2
date_reception_dpe
cout_total_5_usages_energie_n1
cout_ecs_energie_n1
qualite_isolation_plancher_bas
modele_dpe
qualite_isolation_enveloppe
conso_chauffage_generateur_n1_installation_n1
type_energie_n1
emission_ges_eclairage
type_energie_n2
code_postal_ban
emission_ges_ecs
conso_5_usages_ef_energie_n2
conso_5_usages_ef
conso_5_usages_ef_energie_n1
code_insee_ban
deperditions_planchers_bas
conso_5_usages_ep
date_fin_vali

In [24]:
# variables des logements neufs
for col in data_neufs.columns:
    print(col)

emission_ges_5_usages_energie_n1
appartement_non_visite
score_ban
emission_ges_5_usages_par_m2
surface_habitable_immeuble
conso_auxiliaires_ep
complement_adresse_logement
cout_eclairage
cout_auxiliaires
conso_auxiliaires_ef
statut_geocodage
ventilation_posterieure_2012
cout_chauffage
conso_5_usages_par_m2_ep
emission_ges_refroidissement
date_etablissement_dpe
conso_ecs_ef_energie_n1
hauteur_sous_plafond
conso_chauffage_ef
emission_ges_chauffage
nom_commune_ban
conso_5_usages_par_m2_ef
_geopoint
date_visite_diagnostiqueur
type_batiment
conso_chauffage_ef_energie_n1
qualite_isolation_menuiseries
conso_ecs_ep
date_reception_dpe
cout_total_5_usages_energie_n1
cout_ecs_energie_n1
qualite_isolation_plancher_bas
conso_ecs_ef
emission_ges_5_usages
date_derniere_modification_dpe
etiquette_ges
identifiant_ban
modele_dpe
qualite_isolation_enveloppe
ubat_w_par_m2_k
type_energie_n1
emission_ges_eclairage
nom_commune_brut
code_postal_ban
etiquette_dpe
emission_ges_ecs
conso_5_usages_ef
conso_5_usage

In [64]:
# proportions des valeurs manquantes
missing_summary = (
    data_existants.isna().mean() * 100
).reset_index()

missing_summary.columns = ['variable', 'pct_missing']
missing_summary = missing_summary[missing_summary['pct_missing'] > 0]
missing_summary = missing_summary.sort_values('pct_missing', ascending=False)
print(missing_summary)

                                         variable  pct_missing
137                                        _score   100.000000
138                                 categorie_enr    76.063685
144                    surface_habitable_immeuble    48.023631
66   qualite_isolation_plancher_haut_comble_perdu    45.564230
84                             annee_construction    42.469941
..                                            ...          ...
82                             conso_chauffage_ef     0.001471
116                            conso_chauffage_ep     0.001471
106                                  adresse_brut     0.000981
7                            conso_auxiliaires_ef     0.000245
97                                ubat_w_par_m2_k     0.000245

[102 rows x 2 columns]


In [65]:
# proportions des valeurs manquantes
missing_summary = (
    data_neufs.isna().mean() * 100
).reset_index()

missing_summary.columns = ['variable', 'pct_missing']
missing_summary = missing_summary[missing_summary['pct_missing'] > 0]
missing_summary = missing_summary.sort_values('pct_missing', ascending=False)
print(missing_summary)

                          variable  pct_missing
85                          _score   100.000000
57                   categorie_enr    63.059469
6      complement_adresse_logement    26.560279
82          nombre_niveau_logement    20.004987
4       surface_habitable_immeuble     8.704650
50          nombre_niveau_immeuble     8.348086
75              nombre_appartement     5.101608
65                     nom_rue_ban     4.655280
1           appartement_non_visite     4.228899
83      surface_habitable_logement     1.007356
31  qualite_isolation_plancher_bas     0.087271
55                 code_region_ban     0.039895
48            code_departement_ban     0.039895
54                    adresse_brut     0.012467
42                nom_commune_brut     0.002493


In [66]:
# verification des valeurs manquantes sur les adresses des logements neufs
data_neufs['adresse_ban'].isna().sum()

np.int64(0)

In [48]:
data_existants['adresse_ban'].value_counts(dropna=True)

adresse_ban
173 Avenue Barthélémy Buyer 69005 Lyon      1527
134 Rue Challemel Lacour 69008 Lyon          978
44 Rue de Champvert 69005 Lyon               975
241 Avenue du Plateau 69009 Lyon             935
3 Avenue de Ménival 69005 Lyon               772
                                            ... 
34 Chemin de la Vigneronne 69126 Brindas       1
44 Chemin du Guillermy 69126 Brindas           1
258 Chemin des Andres 69126 Brindas            1
63 Rue du Vieux Bourg 69126 Brindas            1
147 Route du Pont du Chene 69126 Brindas       1
Name: count, Length: 83797, dtype: int64

In [41]:
# verification des valeurs manquantes sur les adresses des logements existants
data_existants['adresse_ban'].isna().sum()

np.int64(564)

In [42]:
pct_missing_address = data_existants['adresse_ban'].isna().mean() * 100
print(f"{pct_missing_address:.2f}% des lignes ont une adresse manquante")

0.14% des lignes ont une adresse manquante


In [43]:
mask_adresse_na = data_existants['adresse_ban'].isna()

# Pourcentage de valeurs manquantes dans les autres colonnes parmi ces lignes
cols_a_verifier = ['code_postal_ban', 'nom_commune_ban', 'code_insee_ban']

for col in cols_a_verifier:
    pct_missing = data_existants.loc[mask_adresse_na, col].isna().mean() * 100
    print(f"{col}: {pct_missing:.2f}% manquant parmi les lignes sans adresse")

code_postal_ban: 0.00% manquant parmi les lignes sans adresse
nom_commune_ban: 0.00% manquant parmi les lignes sans adresse
code_insee_ban: 0.00% manquant parmi les lignes sans adresse


In [46]:
data_existants.loc[
    mask_adresse_na & (
        data_existants['code_postal_ban'].notna() |
        data_existants['nom_commune_ban'].notna() |
        data_existants['code_insee_ban'].notna()
    ),
    ['adresse_ban', 'code_postal_ban', 'nom_commune_ban', 'code_insee_ban', 'etiquette_dpe']
]


,adresse_ban,code_postal_ban,nom_commune_ban,code_insee_ban,etiquette_dpe
3212,NaN,69250,FLEURIEU-SUR-SAÔNE,69085,C
3216,NaN,69250,Curis-au-Mont-d'Or,69071,D
3219,NaN,69250,Neuville-sur-Sa?ne,69143,C
3255,NaN,69250,CURIS-AU-MONT-D'OR,69071,D
6457,NaN,69380,DOMMARTIN,69076,A
...,...,...,...,...,...
398330,NaN,69300,CALUIRE ET CUIRE,69034,F
399189,NaN,69500,BRON,69029,D
399196,NaN,69500,BRON,69029,D
399210,NaN,69500,BRON,69029,D


etant données que nous avons un taux bas de valeurs manquantes pour les adresses des logements nous allons les supprimer

In [67]:
data_existants = data_existants.dropna(subset=['adresse_ban']).reset_index(drop=True)

In [68]:
data_existants['adresse_ban'].isna().sum()

np.int64(0)

### uniformisation des données des deux dataframes

In [71]:
# selection des colonnes communes aux deux dataframes
# Colonnes des deux dataframes
import pandas as pd
cols_existants = set(data_existants.columns)
cols_neufs = set(data_neufs.columns)

# Colonnes qui matchent
colonnes_communes = data_existants.columns.intersection(data_neufs.columns).tolist()
print("Colonnes communes :", colonnes_communes)

# Nombre de colonnes communes
nb_colonnes_communes = len(colonnes_communes)
print("Nombre de colonnes communes :", nb_colonnes_communes)

Colonnes communes : ['score_ban', 'conso_auxiliaires_ep', 'cout_eclairage', 'conso_auxiliaires_ef', 'statut_geocodage', 'ventilation_posterieure_2012', 'cout_chauffage', 'conso_5_usages_par_m2_ep', 'date_etablissement_dpe', 'conso_ecs_ef_energie_n1', 'emission_ges_chauffage', 'conso_5_usages_par_m2_ef', 'conso_chauffage_ef_energie_n1', 'qualite_isolation_menuiseries', 'date_reception_dpe', 'cout_total_5_usages_energie_n1', 'cout_ecs_energie_n1', 'qualite_isolation_plancher_bas', 'modele_dpe', 'qualite_isolation_enveloppe', 'type_energie_n1', 'emission_ges_eclairage', 'code_postal_ban', 'emission_ges_ecs', 'conso_5_usages_ef', 'conso_5_usages_ef_energie_n1', 'code_insee_ban', 'conso_5_usages_ep', 'date_fin_validite_dpe', 'code_region_ban', 'version_dpe', 'coordonnee_cartographique_x_ban', 'type_energie_principale_ecs', 'adresse_ban', '_rand', 'production_electricite_pv_kwhep_par_an', 'nombre_niveau_logement', 'surface_habitable_logement', 'cout_ecs', 'emission_ges_5_usages_energie_n1', 

In [72]:
# Colonnes à garder pour chaque dataframe
colonnes_existants = colonnes_communes.copy()
if "annee_construction" in data_existants.columns and "annee_construction" not in colonnes_existants:
    colonnes_existants.append("annee_construction")

# Sélectionner ces colonnes
data_existants_sel = data_existants[colonnes_existants]
data_neufs_sel = data_neufs[colonnes_communes]

# Harmoniser les colonnes avant concat (important)
data_neufs_sel = data_neufs_sel.reindex(columns=colonnes_existants, fill_value=pd.NA)

# Concaténer les deux
data_ademe = pd.concat([data_existants_sel, data_neufs_sel], ignore_index=True)

print(f"\n DataFrame combiné : {data_ademe.shape[0]:,} lignes et {data_ademe.shape[1]} colonnes")
print("Aperçu :")
print(data_ademe.head())


 DataFrame combiné : 447,486 lignes et 85 colonnes
Aperçu :
   score_ban  conso_auxiliaires_ep  cout_eclairage  conso_auxiliaires_ef  \
0       0.36                2136.4            66.4                 928.9   
1       0.25                 882.8            33.9                 383.8   
2       0.27                   1.0            15.3                   0.0   
3       0.27                1734.7            63.1                 754.2   
4       0.41                 866.1            48.8                 376.6   

                   statut_geocodage  ventilation_posterieure_2012  \
0  adresse géocodée ban à l'adresse                             0   
1  adresse géocodée ban à l'adresse                             0   
2  adresse géocodée ban à l'adresse                             0   
3  adresse géocodée ban à l'adresse                             0   
4  adresse géocodée ban à l'adresse                             0   

   cout_chauffage  conso_5_usages_par_m2_ep date_etablissement_dpe 

C:\Users\Surface\AppData\Local\Temp\ipykernel_1988\4004963243.py:14: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  data_ademe = pd.concat([data_existants_sel, data_neufs_sel], ignore_index=True)


In [73]:
data_ademe['annee_construction'].sort_values().unique()

array([1460., 1516., 1522., 1525., 1550., 1580., 1595., 1600., 1610.,
       1621., 1622., 1640., 1650., 1656., 1657., 1660., 1663., 1675.,
       1681., 1686., 1690., 1700., 1720., 1731., 1732., 1740., 1741.,
       1750., 1751., 1760., 1765., 1770., 1779., 1780., 1783., 1785.,
       1789., 1790., 1793., 1800., 1802., 1810., 1812., 1815., 1820.,
       1822., 1824., 1825., 1827., 1830., 1832., 1835., 1836., 1840.,
       1842., 1845., 1846., 1848., 1849., 1850., 1853., 1855., 1856.,
       1858., 1860., 1861., 1862., 1863., 1864., 1865., 1867., 1868.,
       1869., 1870., 1871., 1872., 1873., 1875., 1876., 1878., 1880.,
       1882., 1883., 1884., 1885., 1886., 1887., 1888., 1889., 1890.,
       1891., 1892., 1893., 1894., 1895., 1896., 1897., 1898., 1899.,
       1900., 1901., 1902., 1903., 1904., 1905., 1906., 1907., 1908.,
       1909., 1910., 1911., 1912., 1913., 1914., 1915., 1916., 1917.,
       1918., 1919., 1920., 1921., 1922., 1923., 1924., 1925., 1926.,
       1927., 1928.,

Maintenant nous avons un dataframe unique pour les logements de l'ademe

### Harmonisation des adresses des logements car c'est ce qui servira dans la fusion des données externes que nous allons utiliser ulterieurement

In [74]:
display(data_ademe[['adresse_ban', 'etiquette_dpe']])

,adresse_ban,etiquette_dpe
0,Route de Rochelin 69790 Saint-Clément-de-Vers,E
1,Propières,G
2,290 Chemin du Vernay 69790 Saint-Bonnet-des-Br...,G
3,Route des Proles 69790 Saint-Bonnet-des-Bruyères,C
4,2101 Route du Saint-Rigaud 69790 Propières,G
...,...,...
447481,103 Route de la Joanna 69126 Brindas,A
447482,Chemin de la Madone 69126 Brindas,A
447483,Chemin du Moncel 69126 Brindas,A
447484,60 Chemin des Andres 69126 Brindas,A


In [81]:
import re
import pandas as pd

def is_complete(address):
    if pd.isna(address):
        return False
    address = address.strip()
    pattern = r"^\s*\d{0,4}\s*[A-Za-zÀ-ÖØ-öø-ÿ'’\-\s]+\s+\d{5}\s+[A-Za-zÀ-ÖØ-öø-ÿ'’\-\s]+$"
    return bool(re.match(pattern, address))

# Vérification des adresses valides
print(f"Avant filtrage : {data_ademe.shape}")
data_ademe["adresse_valide"] = data_ademe["adresse_ban"].apply(is_complete)
print(data_ademe["adresse_valide"].value_counts(normalize=True))

# Filtrage
data_ademe_adresse = data_ademe.loc[data_ademe["adresse_valide"]].copy()
print(f"Après filtrage : {data_ademe_adresse.shape}")


Avant filtrage : (447486, 85)
adresse_valide
True     0.970712
False    0.029288
Name: proportion, dtype: float64
Après filtrage : (434380, 86)


In [82]:
data_ademe['adresse_ban'].nunique()

86787

In [83]:
data_ademe_adresse['adresse_ban'].nunique()

83118

In [85]:
data_ademe_adresse.to_csv("ademe_adresses_filtrees.csv", index=False)

In [ ]:
import pandas as pd
import requests
import time
import concurrent.futures
import random

# PARAMÈTRES GÉNÉRAUX 
ENTREE_CSV = "ademe_adresses_filtrees.csv"  
SORTIE_CSV = "ademe_adresses_normalisees.csv" 
BATCH_SIZE = 5000   # nb d’adresses à traiter entre deux sauvegardes
MAX_WORKERS = 10    # threads simultanés
PAUSE = (0.1, 0.3)  # pause aléatoire entre requêtes (évite saturation API)
TIMEOUT = 8         # timeout API
RETRY = 3           # nb max de tentatives par adresse

# === LECTURE ===
print("Lecture des adresses...")
data = pd.read_csv(ENTREE_CSV, dtype=str, low_memory=False)
adresses = data["adresse_ban"].dropna().unique().tolist()
print(f"{len(adresses):,} adresses uniques à normaliser.")

# === FONCTION DE REQUÊTE BAN ===
def normaliser_adresse(adresse):
    if pd.isna(adresse) or not adresse.strip():
        return None

    for tentative in range(RETRY):
        try:
            r = requests.get(
                "https://api-adresse.data.gouv.fr/search/",
                params={"q": adresse, "limit": 1},
                timeout=TIMEOUT
            )
            if r.status_code == 200:
                js = r.json()
                feats = js.get("features", [])
                if not feats:
                    return None
                props = feats[0]["properties"]
                geom = feats[0]["geometry"]["coordinates"]
                return {
                    "adresse_ban": adresse,
                    "adresse_norm": props.get("label"),
                    "score": props.get("score"),
                    "latitude": geom[1],
                    "longitude": geom[0]
                }
            elif r.status_code == 429:
                # trop de requêtes : pause exponentielle
                wait = 2 ** tentative + random.random()
                print(f" 429 Too Many Requests – pause {wait:.1f}s")
                time.sleep(wait)
            else:
                print(f" HTTP {r.status_code} pour {adresse}")
                return None
        except Exception as e:
            print(f" Erreur tentative {tentative+1}/{RETRY} : {e}")
            time.sleep(0.5)
    return None

# === TRAITEMENT PAR LOTS ===
resultats = []
total = len(adresses)

for start in range(0, total, BATCH_SIZE):
    fin = min(start + BATCH_SIZE, total)
    batch = adresses[start:fin]
    print(f" Traitement des adresses {start:,} → {fin:,}...")

    with concurrent.futures.ThreadPoolExecutor(max_workers=MAX_WORKERS) as executor:
        futures = [executor.submit(normaliser_adresse, adr) for adr in batch]
        for i, f in enumerate(concurrent.futures.as_completed(futures), 1):
            res = f.result()
            if res:
                resultats.append(res)
            if i % 500 == 0:
                print(f"  → {i}/{len(batch)} terminées")
            time.sleep(random.uniform(*PAUSE))

    # sauvegarde intermédiaire
    df_partiel = pd.DataFrame(resultats)
    df_partiel.to_csv(SORTIE_CSV, index=False)
    print(f" Sauvegarde intermédiaire ({len(df_partiel):,} lignes)")

print("\n Normalisation terminée !")
df_final = pd.DataFrame(resultats)
df_final.to_csv(SORTIE_CSV, index=False)
print(f"Fichier final sauvegardé : {SORTIE_CSV} ({len(df_final):,} adresses)")


Lecture des adresses...
83,118 adresses uniques à normaliser.
🧭 Traitement des adresses 0 → 5,000...
  → 500/5000 terminées
  → 1000/5000 terminées
  → 1500/5000 terminées
  → 2000/5000 terminées
  → 2500/5000 terminées
  → 3000/5000 terminées
  → 3500/5000 terminées
  → 4000/5000 terminées
  → 4500/5000 terminées
  → 5000/5000 terminées
💾 Sauvegarde intermédiaire (5,000 lignes)
🧭 Traitement des adresses 5,000 → 10,000...
  → 500/5000 terminées
  → 1000/5000 terminées
⚠️ HTTP 504 pour 16 Boulevard Jean XXIII 69008 Lyon
⚠️ HTTP 504 pour 31 Rue Saint Maurice 69008 Lyon
  → 1500/5000 terminées
  → 2000/5000 terminées
  → 2500/5000 terminées
  → 3000/5000 terminées
  → 3500/5000 terminées
  → 4000/5000 terminées
  → 4500/5000 terminées
  → 5000/5000 terminées
💾 Sauvegarde intermédiaire (9,998 lignes)
🧭 Traitement des adresses 10,000 → 15,000...
  → 500/5000 terminées
  → 1000/5000 terminées
  → 1500/5000 terminées
  → 2000/5000 terminées
  → 2500/5000 terminées
  → 3000/5000 terminées
  → 

In [87]:
df_final.head()

,adresse_ban,adresse_norm,score,latitude,longitude
0,290 Chemin du Vernay 69790 Saint-Bonnet-des-Br...,290 Chemin du Vernay 69790 Saint-Bonnet-des-Br...,0.936032,46.266415,4.502530
1,172 Chemin de la Riviere 69790 Saint-Igny-de-Vers,172 Chemin de la Rivière 69790 Saint-Igny-de-Vers,0.942504,46.236696,4.441573
2,Route de Saint Bonnet des Bruyères 69790 Aigue...,Route de Saint Bonnet des Bruyères 69790 Aigue...,0.943299,46.277197,4.434760
3,Route des Proles 69790 Saint-Bonnet-des-Bruyères,Route des Proles 69790 Saint-Bonnet-des-Bruyères,0.943641,46.274676,4.474832
4,2101 Route du Saint-Rigaud 69790 Propières,2101 route du saint-rigaud 69790 Propières,0.945998,46.198515,4.457319


## fusion des données avec les adresses normalisées

In [88]:
import pandas as pd

# Exemple : lecture des deux fichiers
data_ademe_adresse = pd.read_csv("ademe_adresses_filtrees.csv", dtype=str)
df_norm = pd.read_csv("ademe_adresses_normalisees.csv", dtype=str)

# Vérification rapide
print(f"Données ADEME : {data_ademe_adresse.shape}")
print(f"Données normalisées : {df_norm.shape}")

# Fusion sur la clé commune : "adresse_ban"
data_ademe_f = data_ademe_adresse.merge(
    df_norm,
    on="adresse_ban",
    how="left",        
    validate="m:1"     
)

print(f"Fusion terminée : {data_ademe_f.shape}")
print(data_ademe_f[["adresse_ban", "adresse_norm", "score", "latitude", "longitude"]].head())

# Sauvegarde
data_ademe_f.to_csv("ademe_adresse_geo.csv", index=False)


Données ADEME : (434380, 86)
Données normalisées : (83093, 5)
Fusion terminée : (434380, 90)
                                         adresse_ban  \
0      Route de Rochelin 69790 Saint-Clément-de-Vers   
1  290 Chemin du Vernay 69790 Saint-Bonnet-des-Br...   
2   Route des Proles 69790 Saint-Bonnet-des-Bruyères   
3         2101 Route du Saint-Rigaud 69790 Propières   
4   Chemin du Vernay 69790 Saint-Bonnet-des-Bruyères   

                                        adresse_norm               score  \
0      Route de Rochelin 69790 Saint-Clément-de-Vers  0.9421972727272726   
1  290 Chemin du Vernay 69790 Saint-Bonnet-des-Br...  0.9360318181818181   
2   Route des Proles 69790 Saint-Bonnet-des-Bruyères   0.943640909090909   
3         2101 route du saint-rigaud 69790 Propières  0.9459981818181816   
4   Chemin du Vernay 69790 Saint-Bonnet-des-Bruyères  0.9360318181818181   

    latitude longitude  
0  46.228911  4.399445  
1  46.266415   4.50253  
2  46.274676  4.474832  
3  46.198515 

In [89]:
data_ademe_f.shape

(434380, 90)

In [112]:
print(list(data_ademe_f.columns))

['score_ban', 'conso_auxiliaires_ep', 'cout_eclairage', 'conso_auxiliaires_ef', 'statut_geocodage', 'ventilation_posterieure_2012', 'cout_chauffage', 'conso_5_usages_par_m2_ep', 'date_etablissement_dpe', 'conso_ecs_ef_energie_n1', 'emission_ges_chauffage', 'conso_5_usages_par_m2_ef', 'conso_chauffage_ef_energie_n1', 'qualite_isolation_menuiseries', 'date_reception_dpe', 'cout_total_5_usages_energie_n1', 'cout_ecs_energie_n1', 'qualite_isolation_plancher_bas', 'modele_dpe', 'qualite_isolation_enveloppe', 'type_energie_n1', 'emission_ges_eclairage', 'code_postal_ban', 'emission_ges_ecs', 'conso_5_usages_ef', 'conso_5_usages_ef_energie_n1', 'code_insee_ban', 'conso_5_usages_ep', 'date_fin_validite_dpe', 'code_region_ban', 'version_dpe', 'coordonnee_cartographique_x_ban', 'type_energie_principale_ecs', 'adresse_ban', '_rand', 'production_electricite_pv_kwhep_par_an', 'nombre_niveau_logement', 'surface_habitable_logement', 'cout_ecs', 'emission_ges_5_usages_energie_n1', 'emission_ges_5_usag

In [114]:
colonnes_choisies = [
    # Identification bâtiment
    "date_reception_dpe", "conso_auxiliaires_ef", "cout_eclairage","cout_chauffage","conso_5_usages_par_m2_ef","emission_ges_chauffage","modele_dpe","type_energie_n1",
    "emission_ges_eclairage","code_postal_ban","emission_ges_ecs","conso_5_usages_ef","surface_habitable_logement","cout_ecs","emission_ges_5_usages_par_m2","cout_auxiliaires",
    "emission_ges_refroidissement","conso_chauffage_ef","type_batiment","conso_ecs_ef","emission_ges_5_usages","etiquette_ges","etiquette_dpe","cout_total_5_usages","conso_refroidissement_ef",
    "conso_eclairage_ef","cout_refroidissement","type_energie_principale_chauffage","numero_dpe","numero_dpe", "adresse_ban", "adresse_norm", "latitude", "longitude",
]

In [115]:
# selection des colonnes de l'ademe
ademe_final = data_ademe_f[colonnes_choisies]
ademe_final.shape

(434380, 34)

In [116]:
pd.options.display.max_columns = None
ademe_final.head()

,date_reception_dpe,conso_auxiliaires_ef,cout_eclairage,cout_chauffage,conso_5_usages_par_m2_ef,emission_ges_chauffage,modele_dpe,type_energie_n1,emission_ges_eclairage,code_postal_ban,emission_ges_ecs,conso_5_usages_ef,surface_habitable_logement,cout_ecs,emission_ges_5_usages_par_m2,cout_auxiliaires,emission_ges_refroidissement,conso_chauffage_ef,type_batiment,conso_ecs_ef,emission_ges_5_usages,etiquette_ges,etiquette_dpe,cout_total_5_usages,conso_refroidissement_ef,conso_eclairage_ef,cout_refroidissement,type_energie_principale_chauffage,numero_dpe,numero_dpe,adresse_ban,adresse_norm,latitude,longitude
0,2022-05-14,928.9,66.4,2164.5,203.0,7671.2,DPE 3CL 2021 méthode logement,Fioul domestique,17.1,69790,759.3,27197.3,133.6,214.2,63.0,248.2,0.0,23676.5,maison,2343.4,8507.0,E,E,2693.3,0.0,248.5,0.0,Fioul domestique,2269E1049302M,2269E1049302M,Route de Rochelin 69790 Saint-Clément-de-Vers,Route de Rochelin 69790 Saint-Clément-de-Vers,46.228911,4.399445
1,2021-11-24,0.0,15.3,2384.4,470.0,1286.4,DPE 3CL 2021 méthode logement,Bois – Bûches,6.4,69790,87.1,23582.8,50.1,219.7,27.0,0.0,0.0,22149.8,maison,1339.8,1379.9,C,G,2619.4,0.0,93.2,0.0,Électricité,2169E0761678Z,2169E0761678Z,290 Chemin du Vernay 69790 Saint-Bonnet-des-Br...,290 Chemin du Vernay 69790 Saint-Bonnet-des-Br...,46.266415,4.50253
2,2021-08-21,754.2,63.1,1338.6,51.0,517.7,DPE 3CL 2021 méthode logement,Électricité,21.3,69790,59.6,8533.5,166.2,187.2,3.0,154.0,0.0,6553.6,maison,916.6,646.9,A,C,1743.0,0.0,309.1,0.0,Électricité,2169E0199264R,2169E0199264R,Route des Proles 69790 Saint-Bonnet-des-Bruyères,Route des Proles 69790 Saint-Bonnet-des-Bruyères,46.274676,4.474832
3,2021-11-30,376.6,48.8,4588.1,394.0,16260.5,DPE 3CL 2021 méthode logement,Fioul domestique,17.2,69790,141.1,52983.8,134.2,424.1,122.0,73.6,0.0,50186.8,maison,2170.8,16442.9,G,G,5134.6,0.0,249.7,0.0,Électricité,2169E0806561G,2169E0806561G,2101 Route du Saint-Rigaud 69790 Propières,2101 route du saint-rigaud 69790 Propières,46.198515,4.457319
4,2021-09-02,291.9,54.1,3193.8,358.0,11319.1,DPE 3CL 2021 méthode logement,Électricité,13.4,69790,120.1,37269.5,104.1,516.6,110.0,81.6,0.0,34935.6,maison,1848.4,11471.3,G,G,3846.1,0.0,193.5,0.0,Électricité,2169E0251506Z,2169E0251506Z,Chemin du Vernay 69790 Saint-Bonnet-des-Bruyères,Chemin du Vernay 69790 Saint-Bonnet-des-Bruyères,46.267745,4.501981


In [117]:
ademe_final.to_csv("donnees_ademe_finales_69.csv", index=False)

# 2e partie : données ENEDIS

In [90]:
# import des données enedis
donnees_enedis = pd.read_csv("consommation_annuelle_residentielle_par_adresse.csv", sep=";")
donnees_enedis.head()

C:\Users\Surface\AppData\Local\Temp\ipykernel_1988\1911998734.py:2: DtypeWarning: Columns (1,15) have mixed types. Specify dtype option on import or set low_memory=False.
  donnees_enedis = pd.read_csv("consommation_annuelle_residentielle_par_adresse.csv", sep=";")


,Année,Code IRIS,Nom IRIS,Numéro de voie,Indice de répétition,Type de voie,Libellé de voie,Code Commune,Nom Commune,Segment de client,Nombre de logements,Consommation annuelle totale de l'adresse (MWh),Consommation annuelle moyenne par logement de l'adresse (MWh),Consommation annuelle moyenne de la commune (MWh),Adresse,Code EPCI,Code Département,Code Région,Tri des adresses
0,2018,920040601,Renoir i,8.0,NaN,ALLEE,VISCONTI,92004,ASNIERES-SUR-SEINE,RESIDENTIEL,10,18.925,1.893,3.346,8 ALLEE VISCONTI,200054781.0,92.0,11.0,420021
1,2018,920040601,Renoir i,7.0,NaN,ALLEE,VISCONTI,92004,ASNIERES-SUR-SEINE,RESIDENTIEL,10,19.579,1.958,3.346,7 ALLEE VISCONTI,200054781.0,92.0,11.0,420022
2,2018,920040601,Renoir i,6.0,NaN,ALLEE,VISCONTI,92004,ASNIERES-SUR-SEINE,RESIDENTIEL,10,18.886,1.889,3.346,6 ALLEE VISCONTI,200054781.0,92.0,11.0,420023
3,2018,920040604,Metro,184.0,NaN,BOULEVARD,VOLTAIRE,92004,ASNIERES-SUR-SEINE,RESIDENTIEL,29,35.316,1.218,3.346,184 BOULEVARD VOLTAIRE,200054781.0,92.0,11.0,420036
4,2018,920040604,Metro,154.0,NaN,BOULEVARD,VOLTAIRE,92004,ASNIERES-SUR-SEINE,RESIDENTIEL,12,28.286,2.357,3.346,154 BOULEVARD VOLTAIRE,200054781.0,92.0,11.0,420043


In [100]:
donnees_enedis_69 = donnees_enedis[donnees_enedis['Code Département'] == 69]
donnees_enedis_69.head()
donnees_enedis_69.to_csv("donnees_enedis_69.csv", index=False, encoding="utf-8")

In [98]:
donnees_enedis_69['Adresse'].value_counts()

Adresse
1 RUE DU 8 MAI 1945           32
2 RUE VICTOR HUGO             29
4 RUE DE LA REPUBLIQUE        28
20 RUE DE LA REPUBLIQUE       28
3 RUE DU 8 MAI 1945           27
                              ..
11 RUE DE THIZY                1
5 RUE PROFESSEUR TAVERNIER     1
2 RUE FRANCOIS GENIN           1
1 E QUAI GEORGES LEVY          1
26 RUE DE LA CHAUX             1
Name: count, Length: 29040, dtype: int64

### normalisation des adresses de enedis pour faciliter plus tard la jointure avec les données de l'ademe

In [101]:
import pandas as pd
import requests
import time
import random
import json
from concurrent.futures import ThreadPoolExecutor, as_completed
from pathlib import Path

#   PARAMÈTRES GÉNÉRAUX

INPUT_FILE = "donnees_enedis_69.csv"
OUTPUT_FILE = "donnees_enedis_69_normalise.csv"
CACHE_FILE = "cache_ban.json"
MAX_WORKERS = 10  # threads simultanés
SAVE_EVERY = 2000  # sauvegarde automatique tous les X adresses
SLEEP_BETWEEN = (0.05, 0.2)  # pour ne pas saturer l'API

#  CHARGEMENT DES DONNÉES

print("Chargement des données Enedis...")
enedis = pd.read_csv(INPUT_FILE, sep=",", dtype=str)
for col in ["Numéro de voie", "Type de voie", "Libellé de voie", "Code Commune", "Nom Commune"]:
    enedis[col] = enedis[col].fillna("").str.strip()

# Construction de l’adresse à normaliser
enedis["adresse_combinee"] = (
    enedis["Numéro de voie"] + " " +
    enedis["Type de voie"] + " " +
    enedis["Libellé de voie"] + ", " +
    enedis["Code Commune"] + " " +
    enedis["Nom Commune"]
).str.replace(r"\s+", " ", regex=True).str.strip()

print(f"{len(enedis):,} adresses à traiter")

#  GESTION DU CACHE

cache = {}
if Path(CACHE_FILE).exists():
    print("Cache local trouvé, chargement...")
    cache = json.loads(Path(CACHE_FILE).read_text())

def save_cache():
    Path(CACHE_FILE).write_text(json.dumps(cache))

# FONCTION BAN AVEC RETRY

def normaliser_adresse_ban(adresse):
    adresse = adresse.strip()
    if not adresse:
        return None

    # Si déjà dans le cache
    if adresse in cache:
        return cache[adresse]

    url = "https://api-adresse.data.gouv.fr/search/"
    params = {"q": adresse, "limit": 1}

    for attempt in range(3):
        try:
            r = requests.get(url, params=params, timeout=8)
            if r.status_code == 200:
                data = r.json()
                if data["features"]:
                    feat = data["features"][0]
                    props = feat["properties"]
                    geom = feat["geometry"]["coordinates"]
                    res = {
                        "adresse_norm": props.get("label"),
                        "score": props.get("score"),
                        "latitude": geom[1],
                        "longitude": geom[0],
                        "code_insee": props.get("citycode"),
                        "code_postal": props.get("postcode"),
                    }
                    cache[adresse] = res
                    time.sleep(random.uniform(*SLEEP_BETWEEN))
                    return res
            time.sleep(0.2)
        except requests.RequestException:
            time.sleep(0.5)
    cache[adresse] = None
    return None

# ⚡  MULTITHREADING

results = []
to_process = enedis["adresse_combinee"].unique().tolist()
print(f"Lancement du géocodage BAN sur {len(to_process):,} adresses uniques...")

with ThreadPoolExecutor(max_workers=MAX_WORKERS) as executor:
    futures = {executor.submit(normaliser_adresse_ban, adr): adr for adr in to_process}
    done_count = 0
    for future in as_completed(futures):
        done_count += 1
        if done_count % SAVE_EVERY == 0:
            save_cache()
            print(f"{done_count:,} adresses traitées — cache sauvegardé")
        if done_count % 500 == 0:
            print(f"Progression : {done_count:,}/{len(to_process):,}")
    save_cache()

print("Géocodage terminé !")


#  CRÉATION DU DATAFRAME FINAL

# Conversion du cache en DataFrame
norm_data = []
for adr, res in cache.items():
    if res:
        res["adresse_combinee"] = adr
        norm_data.append(res)

enedis_norm = pd.DataFrame(norm_data)

# Jointure avec les données d’origine
enedis_geo = enedis.merge(enedis_norm, on="adresse_combinee", how="left")

# Sauvegarde finale
enedis_geo.to_csv(OUTPUT_FILE, index=False)
print(f" Données enrichies sauvegardées dans {OUTPUT_FILE}")


Chargement des données Enedis...
165,267 adresses à traiter
Lancement du géocodage BAN sur 52,231 adresses uniques...
Progression : 500/52,231
Progression : 1,000/52,231
Progression : 1,500/52,231
2,000 adresses traitées — cache sauvegardé
Progression : 2,000/52,231
Progression : 2,500/52,231
Progression : 3,000/52,231
Progression : 3,500/52,231
4,000 adresses traitées — cache sauvegardé
Progression : 4,000/52,231
Progression : 4,500/52,231
Progression : 5,000/52,231
Progression : 5,500/52,231
6,000 adresses traitées — cache sauvegardé
Progression : 6,000/52,231
Progression : 6,500/52,231
Progression : 7,000/52,231
Progression : 7,500/52,231
8,000 adresses traitées — cache sauvegardé
Progression : 8,000/52,231
Progression : 8,500/52,231
Progression : 9,000/52,231
Progression : 9,500/52,231
10,000 adresses traitées — cache sauvegardé
Progression : 10,000/52,231
Progression : 10,500/52,231
Progression : 11,000/52,231
Progression : 11,500/52,231
12,000 adresses traitées — cache sauvegardé

In [102]:
enedis_geo.head()

,Année,Code IRIS,Nom IRIS,Numéro de voie,Indice de répétition,Type de voie,Libellé de voie,Code Commune,Nom Commune,Segment de client,...,Code Département,Code Région,Tri des adresses,adresse_combinee,adresse_norm,score,latitude,longitude,code_insee,code_postal
0,2018,690400101,Centre,46.0,NaN,AVENUE,DE MONTLOUIS,69040,CHAMPAGNE-AU-MONT-D'OR,RESIDENTIEL,...,69.0,84.0,468802,"46.0 AVENUE DE MONTLOUIS, 69040 CHAMPAGNE-AU-M...",Avenue de Montlouis 69410 Champagne-au-Mont-d'Or,0.706494,45.795658,4.785021,69040,69410
1,2018,690400101,Centre,34.0,NaN,AVENUE,DE MONTLOUIS,69040,CHAMPAGNE-AU-MONT-D'OR,RESIDENTIEL,...,69.0,84.0,468804,"34.0 AVENUE DE MONTLOUIS, 69040 CHAMPAGNE-AU-M...",Avenue de Montlouis 69410 Champagne-au-Mont-d'Or,0.706494,45.795658,4.785021,69040,69410
2,2018,690400102,Contour,13.0,NaN,RUE,JEAN CLAUDE BARTET,69040,CHAMPAGNE-AU-MONT-D'OR,RESIDENTIEL,...,69.0,84.0,468810,"13.0 RUE JEAN CLAUDE BARTET, 69040 CHAMPAGNE-A...",Rue Jean-Claude Bartet 69410 Champagne-au-Mont...,0.714307,45.791613,4.794612,69040,69410
3,2018,690400101,Centre,8.0,NaN,RUE,JOANNES CHOL,69040,CHAMPAGNE-AU-MONT-D'OR,RESIDENTIEL,...,69.0,84.0,468814,"8.0 RUE JOANNES CHOL, 69040 CHAMPAGNE-AU-MONT-...",Rue Joannès Chol 69410 Champagne-au-Mont-d'Or,0.702386,45.796866,4.791910,69040,69410
4,2018,690400101,Centre,10.0,B,AVENUE,LANESSAN,69040,CHAMPAGNE-AU-MONT-D'OR,RESIDENTIEL,...,69.0,84.0,468822,"10.0 AVENUE LANESSAN, 69040 CHAMPAGNE-AU-MONT-...",Avenue de Lanessan 69410 Champagne-au-Mont-d'Or,0.688775,45.795725,4.791754,69040,69410


In [103]:
enedis_geo.shape

(165267, 26)

In [104]:
enedis_geo.columns

Index(['Année', 'Code IRIS', 'Nom IRIS', 'Numéro de voie',
       'Indice de répétition', 'Type de voie', 'Libellé de voie',
       'Code Commune', 'Nom Commune', 'Segment de client',
       'Nombre de logements',
       'Consommation annuelle totale de l'adresse (MWh)',
       'Consommation annuelle moyenne par logement de l'adresse (MWh)',
       'Consommation annuelle moyenne de la commune (MWh)', 'Adresse',
       'Code EPCI', 'Code Département', 'Code Région', 'Tri des adresses',
       'adresse_combinee', 'adresse_norm', 'score', 'latitude', 'longitude',
       'code_insee', 'code_postal'],
      dtype='object')

In [106]:
# selection de colonnes des données enedis
colonnes_choisies = [
    "Année", "adresse_norm", "score", "latitude", "longitude", "code_insee", "code_postal" , "Code Département", "Nombre de logements", "Consommation annuelle totale de l'adresse (MWh)", "Consommation annuelle moyenne par logement de l'adresse (MWh)",
    "Consommation annuelle moyenne de la commune (MWh)"
]

In [107]:
donnees_enedis_selectionnees = enedis_geo[colonnes_choisies]
donnees_enedis_selectionnees.head()

,Année,adresse_norm,score,latitude,longitude,code_insee,code_postal,Code Département,Nombre de logements,Consommation annuelle totale de l'adresse (MWh),Consommation annuelle moyenne par logement de l'adresse (MWh),Consommation annuelle moyenne de la commune (MWh)
0,2018,Avenue de Montlouis 69410 Champagne-au-Mont-d'Or,0.706494,45.795658,4.785021,69040,69410,69.0,37,123.347,3.334,4.391
1,2018,Avenue de Montlouis 69410 Champagne-au-Mont-d'Or,0.706494,45.795658,4.785021,69040,69410,69.0,12,62.83,5.236,4.391
2,2018,Rue Jean-Claude Bartet 69410 Champagne-au-Mont...,0.714307,45.791613,4.794612,69040,69410,69.0,16,103.763,6.485,4.391
3,2018,Rue Joannès Chol 69410 Champagne-au-Mont-d'Or,0.702386,45.796866,4.791910,69040,69410,69.0,12,27.896,2.325,4.391
4,2018,Avenue de Lanessan 69410 Champagne-au-Mont-d'Or,0.688775,45.795725,4.791754,69040,69410,69.0,18,42.015,2.334,4.391


In [118]:
donnees_enedis_selectionnees.to_csv("donnees_enedis_finales_69.csv", index=False)

### données finales

ici on merge les données finales de Ademe et Enedis pour la suite de nos analyses

In [3]:
ademe_final = pd.read_csv("donnees_ademe_finales_69.csv")
ademe_final.shape

(434380, 34)

In [19]:
ademe_final['date_reception_dpe'].sort_values().unique()

array(['2021-07-01', '2021-07-02', '2021-07-03', ..., '2025-10-16',
       '2025-10-17', '2025-10-20'], shape=(1570,), dtype=object)

In [5]:
donnees_enedis_selectionnees = pd.read_csv("donnees_enedis_finales_69.csv")
donnees_enedis_selectionnees.shape

(165267, 12)

In [ ]:
nb_doublons = donnees_enedis_selectionnees[["Année", "adresse_norm", "latitude", "longitude", "code_insee", "code_postal" , "Code Département", "Nombre de logements", "Consommation annuelle totale de l'adresse (MWh)", "Consommation annuelle moyenne par logement de l'adresse (MWh)",
    "Consommation annuelle moyenne de la commune (MWh)"]].duplicated().sum()
print(f"Nombre de doublons exacts (toutes colonnes identiques) : {nb_doublons}")


Nombre de doublons exacts (toutes colonnes identiques) : 7


In [17]:
nb_doublons = ademe_final.duplicated().sum()
print(f"Nombre de doublons exacts (toutes colonnes identiques) : {nb_doublons}")


Nombre de doublons exacts (toutes colonnes identiques) : 0


In [ ]:
print("Doublons ADEME :", ademe_final["adresse_norm"].duplicated().sum())
print("Doublons ENEDIS :", donnees_enedis_selectionnees["adresse_norm", "Année"].duplicated().sum())


Doublons ADEME : 353420
Doublons ENEDIS : 160888


In [26]:
ademe_final['adresse_norm'].nunique()

80959

In [27]:
donnees_enedis_selectionnees['adresse_norm'].nunique()

4378

In [28]:
4378/80959

0.054076754900628715